# 1. Introduction

In [36]:
import os
import torch
import torch.nn as nn
from scripts.custom_dataset import *

# set global state
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_cpu = os.cpu_count()
print(f'This machine run on {device} with {num_cpu} cpus')

This machine run on cuda with 8 cpus


# 2. Dataset and Dataloader

In [ ]:
# set parameters for data loader
folder_path = 'data/loan'
train_ratio = 0.7
batch_size = 64
val_ratio = 0.15

# generate data loader
train_loader, val_loader, test_loader = split_data(folder_path, train_ratio, val_ratio, batch_size)

# 3. Model Architecture

# 4. Loss Function 

# 5. Optimiser

# 6. Training Loop